<a href="https://colab.research.google.com/github/Ankur3107/Machine-Learning-Notes/blob/master/spacy/NER_tagging_%7C_Preprocessing_2_%7C_spacy_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q spacy --upgrade

     |████████████████████████████████| 12.8MB 312kB/s 
     |████████████████████████████████| 460kB 30.3MB/s 
     |████████████████████████████████| 1.1MB 43.1MB/s 
     |████████████████████████████████| 9.1MB 34.6MB/s 
     |████████████████████████████████| 122kB 40.8MB/s 


In [ ]:
!python -m spacy download en_core_web_sm

2021-03-23 13:03:31.579265: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 315kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import pandas as pd
import re
import spacy
from tqdm import tqdm
from spacy.lang.en.stop_words import STOP_WORDS
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
prepositions = [
  "a",
  "abaft",
  "aboard",
  "about",
  "above",
  "absent",
  "across",
  "afore",
  "after",
  "against",
  "along",
  "alongside",
  "amid",
  "amidst",
  "among",
  "amongst",
  "an",
  "anenst",
  "apropos",
  "apud",
  "around",
  "as",
  "aside",
  "astride",
  "at",
  "athwart",
  "atop",
  "barring",
  "before",
  "behind",
  "below",
  "beneath",
  "beside",
  "besides",
  "between",
  "beyond",
  "but",
  "by",
  "circa",
  "concerning",
  "despite",
  "down",
  "during",
  "except",
  "excluding",
  "failing",
  "following",
  "for",
  "forenenst",
  "from",
  "given",
  "in",
  "including",
  "inside",
  "into",
  "lest",
  "like",
  "mid",
  "midst",
  "minus",
  "modulo",
  "near",
  "next",
  "notwithstanding",
  "of",
  "off",
  "on",
  "onto",
  "opposite",
  "out",
  "outside",
  "over",
  "pace",
  "past",
  "per",
  "plus",
  "pro",
  "qua",
  "regarding",
  "round",
  "sans",
  "save",
  "since",
  "than",
  "through",
  "throughout",
  "till",
  "times",
  "to",
  "toward",
  "towards",
  "under",
  "underneath",
  "unlike",
  "until",
  "unto",
  "up",
  "upon",
  "versus",
  "via",
  "vice",
  "with",
  "within",
  "without",
  "worth"
]

In [ ]:
stop_words = list(set(STOP_WORDS) - set(prepositions))

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Temp_Work_nbs/tags_enriched.csv", header=None)
data.columns = ["id", "tag", "count", "unknown_0", "unknown_1"]
data.head()

,id,tag,count,unknown_0,unknown_1
0,2,abuse_of_power,6,1597881182,1
1,3,politics,53,1597881182,1
2,5,fascism,3,1597881182,1
3,25,blumhouse_productions,49,1597881185,3
4,26,dystopian,107,1597881185,3


In [ ]:
data['tag'] = data['tag'].astype(str)
data['tag'] = data['tag'].apply(lambda x: x.lower())
data['tokens'] = data['tag'].apply(lambda x: re.split("[_ ,-]", x))
data.head()

,id,tag,count,unknown_0,unknown_1,tokens
0,2,abuse_of_power,6,1597881182,1,"[abuse, of, power]"
1,3,politics,53,1597881182,1,[politics]
2,5,fascism,3,1597881182,1,[fascism]
3,25,blumhouse_productions,49,1597881185,3,"[blumhouse, productions]"
4,26,dystopian,107,1597881185,3,[dystopian]


In [ ]:
# remove films, flim, movie, movies
year = [str(i) for i in list(range(1900, 2021, 1))]
year_s = [str(i)+'s' for i in list(range(1900, 2021, 1))]
remove_list = ["films", "film", "movie", "movies", "american", "british", "indian", "shot", "set", "english", "hindi", "telugu", 
                 "kannada", "german", "african", "canadian", "tamil","bengali","brazilian", "nigerian", "marathi", "language", ]+year_s+year
def fun1(tokens):
  tokens = [token for token in tokens if token not in remove_list]
  return tokens

def fun2(tokens):
  tokens = [token for token in tokens if token not in stop_words]
  return tokens

In [ ]:
data['tokens'] = data['tokens'].apply(fun1)
data['tokens'] = data['tokens'].apply(fun2)
data['cleaned_tag'] = data['tokens'].apply(lambda x: " ".join(x))
data = data[data['cleaned_tag']!=""]
data = data.drop_duplicates("cleaned_tag")
data.shape

(6408, 7)

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=["ner"])
def do_lemmatization(text):
  doc = nlp(text)
  tokens = []
  for t in doc:
    tokens.append(t.lemma_)
  return " ".join(tokens)

In [ ]:
data['tag_lemma'] = data['cleaned_tag'].progress_apply(do_lemmatization)

100%|██████████| 6408/6408 [00:25<00:00, 256.28it/s]


In [ ]:
data['tag_lemma_tokens'] = data['tag_lemma'].apply(lambda x: x.split(" "))

In [ ]:
patterns = []

for lemma_tokens in data['tag_lemma_tokens'].values:
  lemma_pattern = [{"LEMMA": lt} for lt in lemma_tokens]
  patterns.append({"label":"tag", "pattern":lemma_pattern})

In [ ]:
len(patterns)
patterns[0:5]

[{'label': 'tag',
  'pattern': [{'LEMMA': 'abuse'}, {'LEMMA': 'of'}, {'LEMMA': 'power'}]},
 {'label': 'tag', 'pattern': [{'LEMMA': 'politic'}]},
 {'label': 'tag', 'pattern': [{'LEMMA': 'fascism'}]},
 {'label': 'tag',
  'pattern': [{'LEMMA': 'blumhouse'}, {'LEMMA': 'production'}]},
 {'label': 'tag', 'pattern': [{'LEMMA': 'dystopian'}]}]

In [ ]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy import displacy
import spacy

In [ ]:
from spacy import Language

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=["ner"])
entityruler = EntityRuler(nlp)

#other_pipes = [p for p in nlp.pipe_names if p != "tagger" and p!="tokenizer"]
entityruler.add_patterns(patterns)

@Language.component("entity_extraction_ruler")
def entity_ruler(doc):
    return entityruler(doc)

nlp.add_pipe("entity_extraction_ruler")

<function __main__.entity_ruler>

In [ ]:
#save rules
entityruler.to_disk("/content/drive/MyDrive/Temp_Work_nbs/entity_pattern/")

In [ ]:
# load rules
nlp2 = spacy.load('en_core_web_sm', disable=["ner"])
entityruler = EntityRuler(nlp)
entityruler.from_disk("/content/drive/MyDrive/Temp_Work_nbs/entity_pattern/")

@Language.component("entity_extraction_ruler")
def entity_ruler(doc):
    return entityruler(doc)
nlp2.add_pipe("entity_extraction_ruler")

<function __main__.entity_ruler>

## *you need to call this function to get all tags using ner

In [ ]:
STOP_WORDS_LIST = list(STOP_WORDS)
def get_tags(nlp, text):
  doc = nlp(text)
  tags = []
  for ent in list(doc.ents):
    tokens = ent.text.split(" ")
    i_tag = ""
    for token in tokens:
      if token.lower() not in STOP_WORDS_LIST:
        i_tag+=token+" "
    if i_tag.strip()!="":
      tags.append(i_tag.strip())
  return tags

In [ ]:
text = """
Kashimira police

Glenn Pereira

2012 Accident: Ex-Red Chillies Entertainment employee Charu Khandal’s parents to get Rs 90 lakh as compensation The former Red Chillies employee had been left paralysed in the accident and passed away in 2017.

Commissioner Sadanand Date

A family’s ordeal

Eight stations on Harbour line set to get new skywalks, interconnecting bridges Plan is to create more space as well as multiple entry points at stations whose passenger counts have risen rapidly in recent years.

Kashimira police station

state minorities commission

A tour of Sewri-Nhava Sheva Trans Harbour Link 00:57

The’s bungling has denied’s family the chance to give him a dignified burial. They found the 62-year-old dead a short distance from his home a day after he went missing and cremated him 22 days later, but continued to search for him for over a month without bothering to check their records.Pereira headed out of his Mira Road home to meet officials of a cooperative credit society he had invested with and which is embroiled in a Rs 21.45-crore fraud case on September 20 but didn’t return. “My father always kept my mother informed about his whereabouts whenever he stepped out of home. But that day, there was no word from him,” said Eureka, Pereira’s daughter. When there was no sign of him for hours, Pereira’s wife Rita alerted Eureka and her son, who live separately.The family combed the elderly man’s frequent haunts and tried to retrace the steps he could have taken that day, but the mystery over his disappearance deepened.After a days-long search, they approached the Kashimira police on October 1 and lodged a missing person complaint. But, according to the family, the police sat on their hands until they sought the police commissioner’s intervention last week.They claimed that they had to keep chasing the police to get them to investigate the case. “My mother and my brother visited every hospital they could in search of my father. The investigator, Suresh Shinde, used to ask us if we had any leads! It was only after October 15 did he even start working on the case,” said Eureka, adding that it was only after the family’s insistence that Shinde even questioned officials of the credit society.With the investigation making no headway in over two months, the family sought Mira-Bhayandar-Vasai-Virar Police’s intervention on December 4. Date immediately assigned the case to a new officer, who told the family the next day that Pereira had died on September 21.The police station’s records show that he was found unconscious near his home and declared dead at a hospital that same day. “They showed his photograph to us, and told us that they had filed him away as an unidentified person,” said Eureka. “They claimed they didn’t know it was the same person as my father.” Since Pereira’s body lay unclaimed for days, the police cremated him on October 13. “We have asked for his post-mortem examination report,” said the daughter.The police’s biggest lapse was their failure to check their records of unidentified persons when the family approached them to file a complaint. “Instead of checking the files at his own police station, the officer sent us on a wild goose chase. The police have apologised but how does that make things right? We could have given my father a burial had we got his body,” said an infuriated Eureka.Sanjay Hajare, senior inspector of, said action will be taken against Shinde if he is found out to have been negligent. But, he lay some of the blame with the family as well. “They came very late to lodge the complaint and gave us an old photograph which bore no resemblance with the body that was found. They also told us that Pereira had a habit of staying away from home often. It’s possible that at that point, it was assumed that he would eventually return,” said Hajare.Abraham Mathai, former vice chairman of the, said this case exemplifies the policing system’s shortcomings. “Even this revelation that Pereira had been quietly cremated wouldn’t have emerged had the family not met the Commissioner Sadanand Date. The case would have been relegated to the files of unsolved missing persons.”
"""
get_tags(nlp2, text)

['police',
 'Accident',
 'parents',
 'accident',
 'family',
 'bridges',
 'space',
 'risen',
 'police station',
 'tour',
 'family',
 'home',
 'missing',
 'search',
 'headed',
 'home',
 'meet',
 'society',
 'fraud',
 'father',
 'mother',
 'home',
 'daughter',
 'sign',
 'wife',
 'live',
 'family',
 'elderly',
 'man',
 'haunts',
 'mystery',
 'disappearance',
 'long',
 'search',
 'police',
 'missing person',
 'family',
 'police',
 'police commissioner',
 'chasing',
 'police',
 'mother',
 'brother',
 'visited',
 'hospital',
 'search',
 'father',
 'working',
 'family',
 'society',
 'investigation',
 'making',
 'family',
 'Date',
 'officer',
 'family',
 'died',
 'police station',
 'home',
 'hospital',
 'photograph',
 'father',
 'lay',
 'police',
 'daughter',
 'police',
 'family',
 'police station',
 'officer',
 'chase',
 'police',
 'father',
 'inspector',
 'lay',
 'family',
 'photograph',
 'bore',
 'home',
 'revelation',
 'family',
 'met']

In [ ]:
doc = nlp(text)
displacy.render(doc, jupyter = True, style = "ent")